# Common以下のコード補足

In [2]:
import numpy as np

***
# functions

## softmax関数
xが1次元，2次元の場合に分けて，オーバーフロー対策のために各データから最大値を引く

In [2]:
x = np.random.rand(3, 10)
print(x)
print(x.max(axis=1, keepdims=True))
print(x - x.max(axis=1, keepdims=True))

[[0.60075025 0.33271303 0.17343785 0.13577377 0.46292412 0.09548429
  0.01352433 0.96772625 0.69869944 0.72463471]
 [0.34792022 0.16759107 0.10325365 0.98483311 0.77144506 0.33943378
  0.30998707 0.14994264 0.5835499  0.43993297]
 [0.94991387 0.10350156 0.72904534 0.90166407 0.55088631 0.35083577
  0.13604599 0.04418678 0.35565199 0.82587936]]
[[0.96772625]
 [0.98483311]
 [0.94991387]]
[[-0.366976   -0.63501322 -0.79428841 -0.83195248 -0.50480213 -0.87224197
  -0.95420192  0.         -0.26902681 -0.24309155]
 [-0.63691289 -0.81724204 -0.88157946  0.         -0.21338804 -0.64539933
  -0.67484604 -0.83489047 -0.4012832  -0.54490014]
 [ 0.         -0.84641231 -0.22086852 -0.04824979 -0.39902755 -0.5990781
  -0.81386787 -0.90572708 -0.59426188 -0.1240345 ]]


## cross_entropy_error
- 1データ分
$$
L = - \Sigma_k t_k \log y_k
$$

- バッチ
$$
L = - \frac{1}{N} \Sigma_n \Sigma_k t_{nk} \log y_{nk}
$$

- targetラベルがone-hot-vectorであれば，正解ラベルに対応するlog yを計算するだけ．

In [17]:
# 教師データのone-hot-vectorを正解ラベルのインデックスに変換
t = np.array([[0, 0, 1, 0],
              [1, 0, 0, 0],
              [0, 0, 0, 1]])
t.argmax(axis=1)

array([2, 0, 3])

***
# layers

## SigmoidWithLoss
$$
y = \frac{1}{1 + \exp(-x)}
\\ \\
L = - \{ t \log y + (1 - t) \log(1 - y) \}
$$
- CBOWモデルの高速化(多値分類を二値分類に置き換える)で使用．
- 答えが不正解のときt = 0, 正解のときt = 1が渡される．

In [21]:
y = np.random.rand(5).reshape(5, 1)
# 以下の配列と t = 0 or 1 のラベルをcross_entropy_errorに渡せばよい
print(y)
print(np.c_[1 - y, y])

[[0.59855412]
 [0.6860394 ]
 [0.01790132]
 [0.59828537]
 [0.30156778]]
[[0.40144588 0.59855412]
 [0.3139606  0.6860394 ]
 [0.98209868 0.01790132]
 [0.40171463 0.59828537]
 [0.69843222 0.30156778]]


## Embedding
- CBOWモデルの高速化．入力層->中間層の全結合層MatMulレイヤの代わりに使う．
- MuｔMulでやったことは単語IDのone-hot-vectorと重みW_inの積和で，これはW_inから単語IDに対応するインデックスの行を抜き出すことに等しい

In [25]:
# forwardメソッド
W = np.random.rand(5, 10)  # 重み
contexts = np.array([[0, 2],
                     [1, 3],
                     [2, 4],
                     [3, 0],
                     [4, 2]])  # window_size=2, n=6のコンテキスト

print(f'W: \n {W}')

# コンテキストの数と同じだけEmbeddingレイヤを使ってコンテキストの列ごとに処理する
for i in range(contexts.shape[1]) :
    idx = contexts[:, i]
    out = W[idx]
    print(f'out_{i}: \n {out}')

W: 
 [[0.64842049 0.75373468 0.89759888 0.22947346 0.73813411 0.74858958
  0.17246157 0.34890559 0.96806306 0.79243072]
 [0.2058318  0.39801184 0.47532663 0.24275968 0.9800633  0.57204763
  0.39664609 0.59189766 0.22130369 0.65277418]
 [0.34980481 0.93523668 0.621761   0.59789362 0.50470362 0.63132449
  0.9951341  0.64540727 0.13551466 0.44535372]
 [0.75574076 0.30099359 0.0946908  0.47354068 0.20754353 0.76505764
  0.21567255 0.96124155 0.55462245 0.26425093]
 [0.47646518 0.98103533 0.63031331 0.12753031 0.32248608 0.45891008
  0.25701795 0.01210392 0.98876656 0.68714056]]
out_0: 
 [[0.64842049 0.75373468 0.89759888 0.22947346 0.73813411 0.74858958
  0.17246157 0.34890559 0.96806306 0.79243072]
 [0.2058318  0.39801184 0.47532663 0.24275968 0.9800633  0.57204763
  0.39664609 0.59189766 0.22130369 0.65277418]
 [0.34980481 0.93523668 0.621761   0.59789362 0.50470362 0.63132449
  0.9951341  0.64540727 0.13551466 0.44535372]
 [0.75574076 0.30099359 0.0946908  0.47354068 0.20754353 0.765057

In [26]:
# backwardメソッド



***
# trainer
## Trainer

In [35]:
# データのシャッフル
np.random.permutation(10)

array([6, 1, 3, 4, 0, 9, 7, 5, 2, 8])

In [39]:
# fitメソッド
x = np.random.randn(1000)
data_size = len(x)
batch_size = 32
max_iters = data_size // batch_size  # 31
max_epoch = 10
eval_interval = 20

for epoch in range(max_epoch):
    print(f'--------------epoch: {epoch}--------------')
    print('ここでデータをシャッフル')
    
    for iters in range(max_iters):
        print(f'--------iters: {iters}--------')
        print('ミニバッチを取得')
        print('順伝播で損失計算')
        print('逆伝播で勾配計算')
        print('max_gradを指定していれば勾配クリッピング')
        print('パラメータ更新 (optimizer.update)')
        print('total_lossに損失の値を足し上げる')
        print('loss_countに損失関数の計算回数をカウント')
        
        if (eval_interval is not None) and (iters % eval_interval) == 0:
            print('------eval------')
            print('損失の平均を計算，出力，リスト追加')
            print('total_loss, loss_countを初期化')

--------------epoch: 0--------------
ここでデータをシャッフル
--------iters: 0--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
------eval------
損失の平均を計算，出力，リスト追加
total_loss, loss_countを初期化
--------iters: 1--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 2--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 3--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 4--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 5--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
lo

total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 24--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 25--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 26--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 27--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 28--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 29--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 30--------
ミニバッチを取得

パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
------eval------
損失の平均を計算，出力，リスト追加
total_loss, loss_countを初期化
--------iters: 21--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 22--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 23--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 24--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 25--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 26--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.u